<a href="https://colab.research.google.com/github/ilikot/Robotechnic/blob/master/AStar_Stub_MIPT_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math

In [0]:
"""
SquareGrid class represents agent's environment
0 - cell is traversable
1 - cell is blocked
"""
class SquareGrid:
    #set width, height and fill the grid with zeroes (fully traversable grid)
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.cells = [[0 for _ in range(width)] for _ in range(height)]
    
    #fill the grid given the string representation of the input_map
    #i.e. write "1" to the matrix element that corresponds the obstacle marker "#"
    #everything else except "#" is considered not to be an obstacle 
    def get_map(self, input_map):
        for i in range(self.height):
            for j in range(self.width):
                if input_map[i * self.width + j] == '#':
                    self.cells[i][j] = 1
    
    #out of bounds check
    def in_bounds(self, i, j):
        return 0 <= j < self.width and 0 <= i < self.height
    
    #blocked cell check
    def traversable(self, i, j):
        return not self.cells[i][j]
    
    #printing the grid
    def print(self):
        for c in self.cells:
            print(*c)
    
    
    def neighbors(self, i, j, diagonal=False, cutcorners=False, squeeze=False):
        """
        function, that returns neighbor nodes to the current node (i, j) according to the following parameters:
        diagonal: True, if diagonal moves are allowed
        cutcorners: True, if the agent is allowed to cut corners (only valid id diagoanl is True)
        squeeze: True, if the is also allowed to squeeze between obstacles (only valid when cutcornres is True)
        """
        neighbors = []
        #very primitive code (hard-code). works for cardinal moves only!
        if self.in_bounds(i, j - 1) and self.traversable(i, j - 1):
            neighbors.append((i, j - 1)) #move left
        if self.in_bounds(i - 1, j) and self.traversable(i - 1, j):
            neighbors.append((i - 1, j)) #move up
            
        if self.in_bounds(i, j + 1) and self.traversable(i, j + 1):
            neighbors.append((i, j + 1)) #move right
            
        if self.in_bounds(i + 1, j) and self.traversable(i + 1, j):
            neighbors.append((i + 1, j)) #move down
        
        return neighbors
    
        #!!!TODO!!!
        #Rewrite the code to handle all the parameters (diagonal, curcorners, squeeze).
        #Get read of the hard-code hook, i.e. there should be a loop over all possible neighbors (instead of numerous ifs)
        #and every neighbor-candidate should be checked inside this loop according to the given options,
        #if the checks are passed - the candidate should be appended to neighbors set.
        


In [0]:
#That's how we represent the map and convert it to SquareGrid object
test_map = '''
. . . . . . . . . . . . . b . . . . . . . # # . . . . . . .  
. . . . . . . . . . . . . c . . . . . . . # # . . . . . . . 
. . . . . . . . . . . . . c . . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . c . . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . # # # # # . . . . 
. . . # # . . . . . . . . # # . . . . . . # # # # # . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . .
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . .

'''

#define the SquareGrid object and fill it with a given map
test_grid = SquareGrid(30, 15) #make sure the dimensions match the drawn map
test_grid.get_map(test_map.translate({ ord(c): None for c in ' \n\t\r' })) #remove all whitespaces, tabs etc. 

#validate that map is converted correctly to the SquareGrid object
test_grid.print()

In [0]:
class Node:
    """
    Node class represents a search node
    i, j: coordinates of corresponding grid element
    g: g-value of the node
    h: h-value of the node
    F: f-value of the node
    parent: pointer to the parent-node 
    """
    def __init__(self, i, j, g=math.inf, h=math.inf, F=-1, parent=None):
        self.i = i
        self.j = j
        self.g = g
        if F==-1:
            self.F = self.g + h
        else:
            self.F = F        
        self.parent = parent

#### OPEN list

OPEN is a fringe of the search-space, i.e. it consists of the nodes that have already been generated but not yet expanded.

We should be able to:
i) insert newly generated node to OPEN in case no node with the same (i, j) already resides in OPEN;
ii) update node in case shorter path to the corresponding grid cell is found (update alters g-val, f-val and backpointer, but, obviously, not i, j and h-val);
iii) find the best node, i.e. the node with the lowest f-value.

Let's start withis a very primitive, straight-forward, non-efficent way to implement OPEN list.

In [0]:
class OpenListBasic:
    def __init__(self):
        self.elements = []
    
    #empty should infrom whether the OPEN is exhausted or not (in case it is - the search main loop should be interrupted)
    def empty(self):
        if len(self.elements) != 0:
            return False
        return True
    
    #get is the method that finds the best node (the one with the lowest  f-value),removes it from OPEN and returns it
    def get(self):
        best_F = math.inf
        best_coord = 0
        for i in range(len(self.elements)):
            if self.elements[i].F < best_F:
                best_coord = i
                best_F = self.elements[i].F
                
        best = self.elements.pop(best_coord)
        return best
    
    #put is the method that puts (e.g. inserts or updates) the node to OPEN
    #When implementing it do not forget to handle all the possible situations:
    #- node already in OPEN but the new f-value is better;
    #- node already in OPEN but the new f-value is worse;
    #- node is not in OPEN.
    def put(self, item):
        for coord in range(len(self.elements)):
            if self.elements[coord].i == item.i and self.elements[coord].j == item.j:
                if self.elements[coord].F > item.F:
                    self.elements[coord].F = item.F
                    return
                else:
                    return
                
        
        self.elements.append(item)
        return

In [0]:
#class OpenList:
    #!!!TODO!!!
    #Implement OPEN in a more efficent way compared to OpenListBasic.
    
    #A few directions to try:
    
    #1. Keeping the list sorted, so it takes O(1) to get the best element. Primitive way of "keeping the list sorted"
    #is just explicitly sorting it in accordance with the f-value (i.e. key = f-value) after the new element is appended.
    #It will result in O(n) + O(nlogn) = O(nlogn) performance, i.e. identifying whether a node already exists
    #in the list is O(n) and sorting the list after appending new element is O(nlogn).
    
    #2. Checking the presence of the node in the list and inserting the node in a right place can be combined in
    #one pass so it takes O(n) to put the node in OPEN (and it's still O(1) to get the best node).
    
    #3. One can actually implement OPEN not as a single list but as i sorted lists (i.e. vector of lists),
    #thus checking for duplicates becomes faster. Although getting the best node is now trickier
    #and the memory footprint is higher.
    
    #4. One can think on multi-index structure. To check whether the node is in OPEN a dictionary is a perferct option,
    # as it takes O(1) to check for an element. A seperate structure, e.g. priority_queue (heap) might hold nodes,
    # sorted by f-values, so OPEN.get is O(logN).
    
    

### Heuristics for grid maps

#### Euclidean distance

Straight line distance is the most intuitive thing to think of as it is the true shortest distance on a plane (thus 100% admissable). Calculation involves taking the square root (so it's a "slow" heuristic). Moreover this distance is not accurate for grid-worlds in many cases as the agent is limited to cardinal and diagonal moves only.

In [0]:
def euclidean_heuristic(a_i, a_j, b_i, b_j):
    dx = abs(a_i - b_i)
    dy = abs(a_j - b_j)
    return math.sqrt(dx * dx + dy * dy)

#### Manhattan distance

More accurate heuristic when cardinal moves are allowed. It is not an admissable heuristic if diagonal moves are allowed as well.

In [0]:
def manhattan_distance(a_i, a_j, b_i, b_j):
    #!!!TODO!!!
    #REWRITE THE CODE (it's Euclidean distance now)
    dx = abs(a_i - b_i)
    dy = abs(a_j - b_j)
    return math.sqrt(dx * dx + dy * dy)

#### Diagonal distance

More accurate heuristic when diagonal moves are allowed.

In [0]:
def diagonal_heuristic(a_i, a_j, b_i, b_j):
    #!!!TODO!!!
    #REWRITE THE CODE (it's Euclidean distance now)
    dx = abs(a_i - b_i)
    dy = abs(a_j - b_j)
    return math.sqrt(dx * dx + dy * dy)

__Let's start with A*__

In [0]:
def calculate_heuristic(a_i, a_j, goal_i, goal_j, heuristic_type='euclid'):
    if heuristic_type == 'euclidean':
        return euclidean_heuristic(a_i, a_j, goal_i, goal_j)
    if heuristic_type == 'octile':
        return diagonal_heuristic(a_i, a_j, goal_i, goal_j)
    if heuristic_type == 'manhattan':
        return manhattan_heuristic(a_i, a_j, goal_i, goal_j) 
    
def calculate_cost(a_i, a_j, b_i, b_j):
    return math.sqrt(abs(a_i - b_i) * abs(a_i - b_i) + abs(a_j - b_j) * abs(a_j - b_j))


def search(grid, start_i, start_j, goal_i, goal_j,
           heuristic_type='euclidean',
           heuristic_weight=1,
           diagonal=False, 
           cutcorners=False, 
           squeeze=False):
    
    OPEN = OpenListBasic()
    start_node = Node(start_i, start_j, 0, 
                      heuristic_weight * calculate_heuristic(start_i, start_j, goal_i, goal_j, heuristic_type))
    OPEN.put(start_node)
    CLOSED = dict()
    
    while not OPEN.empty():
        current = OPEN.get() #retrieve the best search node from OPEN
        print(current.i, ' ', current.j, ' ', current.F) #print out the best current node (f-val should increase over the iterations)
        CLOSED[current.i * grid.width + current.j] = current #put the node to CLOSE
        
        if current.i == goal_i and current.j == goal_j:
            print("Path has been found!")
            return current, CLOSED
        
        for (i, j) in grid.neighbors(current.i, current.j):
            if i * grid.width + j not in CLOSED:
                g_cur = current.g + calculate_cost(current.i, current.j, i, j)
                h_cur = calculate_heuristic(i, j, goal_i, goal_j, heuristic_type)
                f_cur = g_cur + heuristic_weight * h_cur
                new_node = Node(i, j, g_cur, h_cur, f_cur, current)
                OPEN.put(new_node)
                
    print("Path NOT found")
    return current, CLOSED

In [0]:
input_map = '''
. . . . . . . . . . . . . . . . . . . . . # # . . . . . . .  
. . . . . . . . . . . . . c . . . . . . . # # . . . . . . . 
. . . . . . . . . . . . . c . . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . c . . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . # # # # # . . . . 
. . . # # . . . . . . . . # # . . . . . . # # # # # . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

'''

In [0]:
grid = SquareGrid(30, 15)
grid.get_map(input_map.translate({ ord(c):None for c in ' \n\t\r' }))

In [0]:
start_i=0
start_j=0
goal_i=10
goal_j=17

%time goal, CLOSED = search(grid, start_i, start_j, goal_i, goal_j)

In [0]:
#some valuable info
print("Path's length (largest g-value in case path NOT found):", goal.g)
print("Number of steps:", len(CLOSED))

In [0]:
def make_path(goal):
    current = goal
    path = []
    while current.parent:
        path.append((current.i, current.j))
        current = current.parent;
    path.append((current.i, current.j))
    return path[::-1]

def print_path(grid, path):
    for i in range(grid.height):
        for j in range(grid.width):
            if (i, j) in path:
                print("*", "", end='')
            else:
                print(grid.cells[i][j], "", end='')
        print("\n", end='')


In [0]:
path = make_path(goal)
print("Found path:", *path, "\n")

print_path(grid, path)

__Bigger maps__

To feel the difference between using various heuristics, heuristic weights, data structures used for OPEN/CLOSED a bigger map is needed.

In [0]:
with open('./maps/1.txt', 'r') as file:
    big_map = file.read()
    
b_grid = SquareGrid(100, 100) #make sure the dimensions match the drawn map
b_grid.get_map(big_map.translate({ ord(c): None for c in ' \n\t\r' })) #remove all whitespaces, tabs etc. 

In [0]:
start_i=10
start_j=0
goal_i=88
goal_j=99

%time goal, CLOSED = search(b_grid, start_i, start_j, goal_i, goal_j)

print("Path's length (largest g-value in case path NOT found):", goal.g)
print("Number of steps:", len(CLOSED))

In [0]:
%time goal, CLOSED = search(b_grid, start_i, start_j, goal_i, goal_j, heuristic_weight=5)

print("Path's length (largest g-value in case path NOT found):", goal.g)
print("Number of steps:", len(CLOSED))

### TODO

1. Look for '!!!TODO!!!' comments and follow them
2. Provide a report (pdf) on your experience on evaluation of A* with different heuristics and heuristics weights on a large map.

Requirments for A* evaluation:
- you should use (e.g. create) your own map
- map should be unique (no maps repetition across students)
- map size: 256 x 256 (min.)
- percent of blocked cells: ~20% (e.g. randomly blocked cells) 
- number of tasks: 50 (min) - 100 (can be larger)
- task generation strategy: any (but you should be able to describe it in a proper way), e.g. random generation of start-goal pairs
- tasks should be unique (no tasks repetition across students)
- At least TWO tests should be carried out:
-- testing (at least two) different heuristics (e.g. Euclidean vs Diagonal)
-- testing (at least three) different weights (e.g. Diagonal heuristic with w=1.1, 1.5, 3)
-- more tests are welcome
- in each test the following indicators should be measured:
-- number of steps
-- path lengths (common sense check: path lenghts should be the same for test1 and should be different for test 2)
- Evaluation results should be presented as plots and (optionally) tables


